**Notebook to visualize the forest**

In [ ]:
import os, sys
import pandas as pd
import sklearn
import numpy as np
import graphviz
import io
import pydotplus
import sklearn.ensemble
import matplotlib as plt
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree
from scipy import misc
path = os.getcwd() + '/data/'

In [ ]:
# concatenate the data

log1 = pd.read_csv(path + 'student_log_{}.csv'.format(1))
initial_col_order = [col.lower() for col in list(log1.columns)]

data = pd.DataFrame()
for i in range(1, 2):
    student_log = pd.read_csv(path + 'student_log_{}.csv'.format(i))
    student_log.columns = [col.lower() for col in student_log]
    data = pd.concat([data, student_log], ignore_index=True)

data = data[initial_col_order]
data.shape
data.head()

training = pd.read_csv(path + 'training_label.csv').drop(['AveCorrect'], axis=1)
training.columns = [col.lower() for col in training.columns]
training.shape

# testing on a little dataset
df = data
df1 = df.merge(training, on='itest_id', how="left")

actions = df1[df1.isstem.notnull()]

print("training set size: ", training.shape[0], "actions for # of students: ", len(actions.itest_id.unique()))
print("sample size useless for training: ", training.shape[0] - len(actions.itest_id.unique()))


# merged training data set
stem = actions[actions.isstem == 1]
nonstem = actions[actions.isstem == 0]

# prepare data
actions_clear = actions.drop(['actionid', 'problemid', 'assignmentid', 'assistmentid', 'starttime', 'endtime'], axis=1)

# dummy columns
nominal_columns = ['sy assistments usage','skill']
dummy_df = pd.get_dummies(actions_clear['problemtype'])
print(dummy_df.columns)
print('0' in dummy_df)
# dummy_df = dummy_df.drop([u'0', u'1'], axis=1)
print('0' in dummy_df)

new_actions = pd.concat([actions_clear, dummy_df], axis=1)
print('0' in new_actions)

nominal_columns = ['sy assistments usage', 'skill']
dummy_df = pd.get_dummies(actions_clear[nominal_columns])
new_actions = pd.concat([new_actions, dummy_df], axis=1)
print('0' in new_actions)
new_actions = new_actions.drop(['sy assistments usage', 'skill', 'problemtype'], axis=1)

target = new_actions['isstem']
new_actions = new_actions.drop(['isstem'], axis=1)

new_actions['ln'] = new_actions['ln'].astype(float)
new_actions['ln-1'] = new_actions['ln-1'].astype(float)

new_actions = new_actions.drop(['sumtime3sdwhen3rowright'], axis=1)


In [1]:
# define function to plot the forest
def show_tree(tree, path):
    f = io.BytesIO()
    export_graphviz(tree, out_file=f, feature_names=features)
    pydotplus.graph_from_dot_data(f.getvalue()).write_png(path)


In [ ]:
# plot forest
i_tree = 0
for tree_in_forest in clf.estimators_:
    if (i_tree <1):
        f = io.BytesIO()
#         tree.export_graphviz(tree_in_forest, out_file=f)
        export_graphviz(tree_in_forest,
                    feature_names=new_actions.columns,
                    filled=True,
                    rounded=True, out_file=f)
    #     os.system('dot -Tpng tree.dot -o tree.png')
        pydotplus.graph_from_dot_data(f.getvalue()).write_png('dtree'+ str(i_tree) +'.png')
        i_tree = i_tree + 1
